In [19]:
from neo4j import GraphDatabase
import pandas as pd

In [5]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [6]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

conn = Neo4jConnection(uri=URI, user=AUTH[0], pwd=AUTH[1])

find/define the research communities

In [8]:
research_communities = [['database', ['data management', 'indexing', 'data modeling', 'big data', 'data processing', 'data storage', 'data querying']],
            ['artificial_intelligence', ['machine learning', 'neural networks', 'deep learning', 'natural language processing', 'computer vision', 'reinforcement learning', 'expert systems', 'knowledge representation', 'genetic algorithms', 'bayesian networks']],
            ['cybersecurity', ['cybersecurity', 'network security', 'information security', 'vulnerability', 'penetration testing', 'threat detection', 'malware analysis', 'security policies', 'risk management', 'cybercrime']]
]

In [14]:
query =''' UNWIND [
                   ['database', ['data management', 'indexing', 'data modeling', 'big data', 'data processing', 'data storage', 'data querying']],
                   ['artificial_intelligence', ['machine learning', 'neural networks', 'deep learning', 'natural language processing', 'computer vision', 'reinforcement learning', 'expert systems', 'knowledge representation', 'genetic algorithms', 'bayesian networks']],
                   ['cybersecurity', ['cybersecurity', 'network security', 'information security', 'vulnerability', 'penetration testing', 'threat detection', 'malware analysis', 'security policies', 'risk management', 'cybercrime']]
                  ] AS rsrch_com
    MERGE (c:Community {name: rsrch_com[0]})
    WITH c, rsrch_com
    UNWIND rsrch_com[1] as keyword
    MERGE (key:Keyword {word: keyword})
    MERGE (key)-[:DEFINES]->(c)
'''

res = conn.query(query=query)

find the conferences and journals related to the database community
(i.e., are specific to the field of databases). Assume that if 90% of the papers published
in a conference/journal contain one of the keywords of the database community we
consider that conference/journal as related to that community.

In [26]:
query =''' MATCH (n:Conference|Journal)
CALL {
    WITH n
    MATCH (n)--()--(p:Paper)--(:Keyword)--(c:Community)
    WHERE c.name = "database"
    WITH n, c, count(distinct p) as num_papers_com
    RETURN num_papers_com 
}
WITH n, num_papers_com
MATCH (n1:Conference|Journal)--()--(p1:Paper)
WHERE n1.id = n.id
WITH n, num_papers_com, count(p1) AS total_num_papers
WHERE 100*toFloat(num_papers_com)/total_num_papers >= 90
RETURN n.id, num_papers_com, total_num_papers, 100*toFloat(num_papers_com)/total_num_papers
'''
res = conn.query(query=query)
df = pd.DataFrame(res, columns=["ID", "Number of Papers per Community", "Total Number of Papers", "ratio"])
df

ID  Number of Papers per Community  Total Number of Papers  ratio
0  92925374                               1                       1  100.0
1  54626690                               1                       1  100.0
2  20325678                               1                       1  100.0